In [2]:
import os
notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
data_dir = os.path.join(root_dir, 'data')

import pandas as pd
import numpy as np


In [3]:
events_df = pd.read_parquet(os.path.join(data_dir, 'event_data.parquet'))
events_df["half"] = round((events_df["quarter"] + 0.01) / 2)
events_df["home_timeout"] = np.where(((events_df["event_id"]==57)&(events_df["home_team_has_ball"]==1))|((events_df["event_id"]==58)&(events_df["home_team_has_ball"]==0)), 1, 0)
events_df["away_timeout"] = np.where(((events_df["event_id"]==57)&(events_df["home_team_has_ball"]==0))|((events_df["event_id"]==58)&(events_df["home_team_has_ball"]==1)), 1, 0)
events_df = events_df.sort_values(["game_code", "nevent"])
events_df["home_timeouts_remaining"] = np.clip(3 - events_df.groupby(["game_code", "half"])["home_timeout"].cumsum(), 0, 3)
events_df["away_timeouts_remaining"] = np.clip(3 - events_df.groupby(["game_code", "half"])["away_timeout"].cumsum(), 0, 3)

In [21]:
play_example = events_df.loc[0]
yes = 1
np.where(play_example["home_team_has_ball"], "xhome_win_", "xaway_win").tolist().join(["go_for_it", "punt", "field_goal_attempt"])

'go_for_itxaway_winpuntxaway_winfield_goal_attempt'

['home_score_0', 'home_score_1', 'home_score_2', 'home_score_3', 'home_score_4', 'home_score_5', 'home_score_6', 'home_score_7', 'home_score_8', 'home_score_9', 'home_score_10', 'home_score_11', 'home_score_12', 'home_score_13', 'home_score_14', 'home_score_15', 'home_score_16', 'home_score_17', 'home_score_18', 'home_score_19', 'home_score_20', 'home_score_21', 'home_score_22', 'home_score_23', 'home_score_24', 'home_score_25', 'home_score_26', 'home_score_27', 'home_score_28', 'home_score_29', 'home_score_30', 'home_score_31', 'home_score_32', 'home_score_33', 'home_score_34', 'home_score_35', 'home_score_36', 'home_score_37', 'home_score_38', 'home_score_39', 'home_score_40', 'home_score_41', 'home_score_42', 'home_score_43', 'home_score_44', 'home_score_45', 'home_score_46', 'home_score_47', 'home_score_48', 'home_score_49', 'home_score_50', 'home_score_51', 'home_score_52', 'home_score_53', 'home_score_54', 'home_score_55', 'home_score_56', 'home_score_57', 'home_score_58', 'home_

In [5]:
events_df[["event_id", "event_name"]].drop_duplicates().sort_values("event_id").to_csv(os.path.join(data_dir, "event_ids.csv"), index=False)


In [3]:
import numpy as np
np.arange(0, 49, 7)

array([ 0,  7, 14, 21, 28, 35, 42])